## 决策树的构造
先前使用的 k-近邻算法可以完成很多分类任务，但它最大的缺点就是无法给出数据的内在含义，决策树的主要优势就在于数据形式非常容易理解。

【重要任务】：理解数据中所蕴含的知识信息，因此决策树可以使用不熟悉的数据集合，并从中提取出一系列规则，这些机器根据数据集创建规则的过程，就是机器学习的过程。
- 优点：计算复杂度不高，输出结果易于理解，对中间值的缺失不敏感，可以处理不相关特征数据。
- 缺点：可能会产生过度拟合问题。
- 适用数据类型：数值型和标称型。

![决策树结构](http://i4.qhimg.com/t01a9c22a13936739de.jpg)

构造决策树时，第一个需要解决的问题是：当前数据集上哪个特征在划分数据分类时起决定性作用。为了找到决定性的特征，划分出最好的结果，我们必须评估每个特征。完成评估之后，原始数据集就被划分为几个数据子集。这些数据子集会分布在第一个决策点的所有分支上。如果某个分支下的数据属于同一类型，则无需进一步对数据集进行分割。如果数据子集内的数据不属于同一类型，则需要重复划分数据子集的过程。划分数据子集的算法和划分原始数据集的方法相同，直到所有具有相同类型的数据均在一个数据子集内。

【伪代码】：创建分支的伪代码 createBranch() 函数。
```
检测数据集中的每个子项是否属于同一分类：
    if so return 类标签
    else
        寻找划分数据集的最好特征
        划分数据集
        创建分支节点
            for 每个划分的子集
                调用函数 createBranch 并增加返回结果到分支节点中
        return 分支节点
```

【决策树的一般流程】：
- 收集数据：可以使用任何方法。
- 准备数据：树构造算法只适用于标称型数据，因此数值型数据必须离散化。
- 分析数据：可以使用任何方法，构造树完成之后，我们应该检查图形是否符合预期。
- 训练算法：构造树的数据结构。
- 测试算法：使用经验树计算错误率。
- 使用算法：此步骤可以适用于任何监督学习算法，而使用决策树可以更好地理解数据的内在含义。

下面我们将使用 ID3 算法构造决策树。

【数据集】：海洋生物数据。

序号 | 不浮出水面是否可以生存 | 是否有脚蹼 | 属于鱼类
---|---|---|---|
1 | 是 | 是 | 是
2 | 是 | 是 | 是
3 | 是 | 否 | 否
4 | 否 | 是 | 否
5 | 否 | 是 | 否

### 信息增益
划分数据集的大原则：将无序的数据变得更加有序。我们可以使用多种方法划分数据集，但是每种方法都有各自的优缺点。组织杂乱无章数据的一种方法就是使用**信息论度量信息**，我们可以在划分数据之前或之后使用信息论量化度量信息的内容。

在划分数据集之前之后信息发生的变化称为信息增益，知道如何计算信息增益，我们就可以计算每个特征值划分数据集获得的信息增益，获得信息增益最高的特征就是最好的选择。

In [6]:
from math import log


def calc_shannon_ent(dataset):
    num_entries = len(dataset)
    label_counts = {}
    for feat_vec in dataset:
        current_label = feat_vec[-1]
        if current_label not in label_counts.keys():
            label_counts[current_label] = 0
        label_counts[current_label] += 1
    shannon_ent = 0.0
    for key in label_counts:
        prob = float(label_counts[key]) / num_entries
        shannon_ent -= prob * log(prob, 2)
    return shannon_ent

1. 计算数据集中实例的总数；
2. 创建一个数据字典，它的键值是最后一列的数值。如果当前键值不存在，则扩展字典并将当前键值加入字典。每个键值都记录了当前类别出现的次数。
3. 使用所有类别的发生频率计算类别出现的频率。

In [2]:
def create_dataset():
    dataset = [
        [1, 1, 'yes'],
        [1, 1, 'yes'],
        [1, 0, 'no'],
        [0, 1, 'no'],
        [0, 1, 'no']
    ]
    labels = ['no surfacing', 'flippers']
    return dataset, labels

In [32]:
my_dat, labels = create_dataset()

In [4]:
my_dat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [7]:
calc_shannon_ent(my_dat)

0.9709505944546686

熵越高，则混合的数据也越多，我们可以在数据集中添加更多的分类，观察熵是如何变化的。这里我们增加第三个名为 maybe 的分类，测试熵的变换。

In [8]:
my_dat[0][-1] = 'maybe'

In [9]:
my_dat

[[1, 1, 'maybe'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [10]:
calc_shannon_ent(my_dat)

1.3709505944546687

### 划分数据集
分类算法除了需要测量信息熵，还需要划分数据集，度量划分数据集的熵，以便判断当前是否正确地划分数据集。我们将对每个特征划分数据集的结果计算一次信息熵，然后判断哪个特征划分数据集是最好的划分方式。想象一个分布在二维空间的数据散点图，需要在数据之间划条线，将它们分成两部分，我们可以按照 x 轴或 y 轴进行划线，划线的依据是划分后的数据集的无序程度最低。

In [11]:
def split_dataset(dataset, axis, value):
    ret_dataset = []
    for feat_vec in dataset:
        if feat_vec[axis] == value:
            reduced_feat_vec = feat_vec[:axis]
            reduced_feat_vec.extend(feat_vec[axis + 1:])
            ret_dataset.append(reduced_feat_vec)
    return ret_dataset

split_dataset() 函数使用了三个输入参数：待划分的数据集、划分数据集的特征、需要返回的特征的值。

【注意】：Python 语言不用考虑内存分配问题。Python 语言在函数中传递的是列表的引用，在函数内部对列表对象的修改，将会影响该列表对象的整个生存周期。为了消除这个不良影响，我们需要在函数的开始声明一个新列表对象。因为该函数代码在同一数据集上被调用多次，为了不修改原始数据集，创建一个新的列表对象。

In [13]:
split_dataset(my_dat, 0, 1)

[[1, 'maybe'], [1, 'yes'], [0, 'no']]

In [25]:
def choose_best_feature_to_split(dataset):
    num_features = len(dataset[0]) - 1
    # 计算整个数据集的原始想农熵
    base_entropy = calc_shannon_ent(dataset)
    best_info_gain, best_feature = 0.0, -1
    for i in range(num_features):
        feat_list = [example[i] for example in dataset]
        # 创建唯一的分类标签列表
        unique_vals = set(feat_list)
        new_entropy = 0.0
        # 计算每种划分方式的信息熵
        for value in unique_vals:
            sub_dataset = split_dataset(dataset, i, value)
            prob = len(sub_dataset) / float(len(dataset))
            new_entropy += prob * calc_shannon_ent(sub_dataset)
        info_gain = base_entropy - new_entropy
        # 计算最好的信息增益
        if info_gain > best_info_gain:
            best_info_gain, best_feature = info_gain, i
    return best_feature

choose_best_feature_to_split() 函数实现选取特征，划分数据集，计算得出最好的划分数据集的特征。

【数据要求】：
- 数据必须是一种由列表元素组成的列表，而且所有的列表元素都要具有相同的数据长度；
- 数据的最后一列或者每个实例的最后一个元素是当前实例的类别标签。

In [26]:
choose_best_feature_to_split(my_dat)

0

### 递归构建决策树
目前我们已经创建了从数据集构造决策树算法所需要的子功能模块，其工作原理如下：得到原始数据集，然后基于最好的属性值划分数据集，由于特征值可能多于两个，因此可能存在大于两个分支的数据集划分。第一次划分之后，数据将被向下传递到树分支的下一个节点，在这个节点上，我们可以再次划分数据。因此我们可以采用递归的原则处理数据集。

【递归结束的条件】：
- 程序遍历完所有划分数据集的属性；
- 每个分支下的所有实例都具有相同的分类。

第一个结束条件使得算法可以终止，我们甚至可以设置算法可以划分的最大分组数目。

如果数据集已经处理了所有属性，但是类标签依然不是唯一的，此时我们需要决定如何定义该叶子结点，在这种情况下，我们通常会采用多数表决的方法决定该叶子结点的分类。

In [28]:
import operator

def majority_cnt(class_list):
    class_count = {}
    for vote in class_list:
        if vote not in class_count.keys():
            class_count[vote] = 0
        class_count[vote] += 1
    sorted_class_count = sorted(class_count.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_class_count[0][0]

majority_cnt() 函数使用分类名称的列表，然后创建键值为 class_list 中唯一值的数据字典，字典对象存储了 class_list 中每个类标签出现的频率，最后利用 operator 操作键值排序字典，并返回出现次数最多的分类名称。

In [29]:
def create_tree(dataset, labels):
    class_list = [example[-1] for example in dataset]
    # 类别完全相同则停止继续划分
    if class_list.count(class_list[0]) == len(class_list):
        return class_list[0]
    # 遍历完所有特征时返回出现次数最多的类别
    if len(dataset[0]) == 1:
        return majority_cnt(class_list)
    # 找寻最好的属性，进行划分
    best_feat = choose_best_feature_to_split(dataset)
    best_feat_label = labels[best_feat]
    # 得到列表包含的所有属性值
    my_tree = {best_feat_label: {}}
    del(labels[best_feat])
    feat_values = [example[best_feat] for example in dataset]
    unique_vals = set(feat_values)
    # 遍历当前选择特征包含的所有属性值，在每个数据集划分上递归调用函数 create_tree()
    for value in unique_vals:
        sub_labels = labels[:]
        # 得到的返回值将被插入到字典变量 my_tree 中。
        my_tree[best_feat_label][value] = create_tree(split_dataset(dataset, best_feat, value), sub_labels)
    return my_tree

【注意】：`sub_labels = labels[:]` 这行代码复制了类标签，并将其存储在新列表变量 sub_labels 中。之所以这样做，是因为在 Python 语言中函数参数是列表类型时，参数是按照引用方式传递的。为了保证每次调用函数 create_tree() 时不改变原始列表的内容，使用新变量 sub_labels 代替原始列表。

In [34]:
my_tree = create_tree(my_dat, labels)

In [35]:
my_tree

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

变量 my_tree 包含很多代表树结构信息的嵌套字典，从左边开始，第一个关键字 no surfacing 是第一个划分数据集的特征名称，该关键字的值也是另一个数据字典。第二个关键字是 no surfacing 特征划分的数据集，这些关键字的值是 no surfacing 节点的子节点。这些值可能是类标签，也可能是另一个数据字典。如果值是类标签，则该子节点是叶子节点；如果值是另一个数据字典，则子节点是一个判断节点，这种格式结构不断重复就构成了整棵树。